# 🧬 UFoE Creator Engine v4 + ESMFold 교차 검증

**법칙 기반 단백질 서열 설계 → ESMFold 접힘 검증**

Author: Young Kang (MCWS) + SYM | 2026-02-08

---

### 핵심 발견 (2026-02-08)

1. **감(G,P,C) = 결절점** — 턴에 76% 위치 (p=0.0016), Precision 70%
2. **구조 우선 설계** — 감 선배치 → 친화도 충전 → v3 대비 25%→80% 통과율
3. **5클러스터 재현** — 16,221개 PDB에서 상극축 × 감밀도 좌표계 확인
4. **피보나치 기각** — Null과 동일 (84.2% vs 81.4%), 정직한 수정 반영

### 이 노트북의 목표

Creator Engine v4로 생성한 서열이 **실제로 접히는지** ESMFold로 검증

```
v4 서열 생성 → ESMFold pLDDT 측정 → 12칸 매트릭스 보존 확인
```

⚠ **GPU 필요**: 런타임 → 런타임 유형 변경 → T4 GPU

---


## 1. 환경 설정

In [ ]:
# HuggingFace Transformers 기반 ESMFold (openfold 불필요)
!pip install -q transformers accelerate

import torch
import numpy as np
import math
import json
from collections import defaultdict, Counter
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Device: {device}")
if device.type == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_mem / 1e9:.1f} GB")
else:
    print("   ⚠ GPU 없음 — ESMFold 실행 불가. 런타임 → T4 GPU로 변경하세요.")


## 2. UFoE 상수 + 곤감리건 분류

In [ ]:
# ══════════════════════════════════════════
# 곤감리건 4타입 분류 (7:3:6:4 = 20)
# ══════════════════════════════════════════
GON_AAS = ['V', 'I', 'L', 'F', 'W', 'M', 'A']   # 소수성 (7)
GAM_AAS = ['C', 'G', 'P']                          # 구조적 제약 (3) — 결절점
RI_AAS  = ['S', 'T', 'N', 'Q', 'H', 'Y']          # 극성 접착 (6)
GEON_AAS = ['K', 'R', 'D', 'E']                    # 전하 (4)

TYPE_MAP = {}
for aa in GON_AAS:  TYPE_MAP[aa] = 'Gon'
for aa in GAM_AAS:  TYPE_MAP[aa] = 'Gam'
for aa in RI_AAS:   TYPE_MAP[aa] = 'Ri'
for aa in GEON_AAS: TYPE_MAP[aa] = 'Geon'

TYPES = ['Gon', 'Gam', 'Ri', 'Geon']
ZONES = ['Center', 'Transition', 'Boundary']

# 4×4 친화도 행렬
AFFINITY = {
    ('Gon','Gon'): 5, ('Gon','Gam'): 2, ('Gon','Ri'): 3, ('Gon','Geon'): 0,
    ('Gam','Gon'): 2, ('Gam','Gam'): 3, ('Gam','Ri'): 4, ('Gam','Geon'): 2,
    ('Ri','Gon'):  3, ('Ri','Gam'):  4, ('Ri','Ri'):  3, ('Ri','Geon'):  4,
    ('Geon','Gon'):0, ('Geon','Gam'):2, ('Geon','Ri'): 4, ('Geon','Geon'):3,
}

# PDB 1,473개 평균 기준 매트릭스
BASELINE_MATRIX = {
    ('Center', 'Gon'):    0.4625,  ('Center', 'Gam'):    0.1366,
    ('Center', 'Ri'):     0.2455,  ('Center', 'Geon'):   0.1534,
    ('Transition', 'Gon'):0.4042,  ('Transition', 'Gam'):0.1365,
    ('Transition', 'Ri'): 0.2546,  ('Transition', 'Geon'):0.2048,
    ('Boundary', 'Gon'):  0.2909,  ('Boundary', 'Gam'):  0.1513,
    ('Boundary', 'Ri'):   0.2747,  ('Boundary', 'Geon'): 0.2830,
}

PHI = (1 + math.sqrt(5)) / 2
CENTER_THRESH = 1 / (PHI ** 2)
TRANS_THRESH = 1 / PHI

print("✅ UFoE 상수 정의 완료")
print(f"   곤:감:리:건 = 7:3:6:4")
print(f"   친화도 행렬: 곤-곤=5 (최대), 곤-건=0 (상극)")
print(f"   감 = 결절점 (G:유연, P:꺾임, C:고정)")


## 3. Creator Engine v4 — 구조 우선 설계

**핵심 변경 (v3 → v4):**
- v3: 랜덤 배치 → 소거 → 25% 통과
- v4: 감 선배치 → 친화도 충전 → 80% 통과

**알고리즘:**
1. 감(G,P,C)을 먼저 배치 — 뼈대
2. 세그먼트별 곤/리/건 배치 — 살
3. 곤-건 최대 거리 원칙 — 상극
4. 인접 친화도 최적화 — 접착


In [ ]:
def generate_signature(target_antagonism):
    """목표 상극축 → 12칸 시그니처 생성."""
    sig = {k: v for k, v in BASELINE_MATRIX.items()}
    
    baseline_antag = (
        (sig[('Center','Gon')] - sig[('Boundary','Gon')]) -
        (sig[('Center','Geon')] - sig[('Boundary','Geon')])
    )
    delta = target_antagonism - baseline_antag
    adj = delta / 4.0
    
    sig[('Center', 'Gon')]    += adj
    sig[('Boundary', 'Gon')]  -= adj
    sig[('Center', 'Geon')]   -= adj
    sig[('Boundary', 'Geon')] += adj
    
    for z in ZONES:
        total = sum(sig[(z, t)] for t in TYPES)
        if abs(total - 1.0) > 0.001:
            sig[(z, 'Ri')] += (1.0 - total)
    
    for key in sig:
        sig[key] = max(0.01, min(0.99, sig[key]))
    
    for z in ZONES:
        total = sum(sig[(z, t)] for t in TYPES)
        for t in TYPES:
            sig[(z, t)] /= total
    
    return sig


def creator_engine_v4(length, target_antagonism, gam_density=0.15, seed=42):
    """
    Creator Engine v4: 구조 우선 설계.
    
    1. 감(G,P,C)을 먼저 배치 (결절점)
    2. 세그먼트별 곤/리/건을 친화도 규칙으로 충전
    3. 곤-건 최대 거리 원칙 적용
    
    Returns: sequence, signature, metadata
    """
    rng = np.random.default_rng(seed)
    
    # ── Step 1: 목표 시그니처 생성 ──
    target_sig = generate_signature(target_antagonism)
    
    # ── Step 2: 전체 타입 개수 결정 ──
    zone_fracs = {'Center': 0.17, 'Transition': 0.41, 'Boundary': 0.42}
    zone_counts = {}
    remaining = length
    for z in ['Center', 'Transition']:
        zone_counts[z] = max(2, round(length * zone_fracs[z]))
        remaining -= zone_counts[z]
    zone_counts['Boundary'] = max(2, remaining)
    
    # Zone × Type 개수
    composition = {}
    for z in ZONES:
        z_total = zone_counts[z]
        probs = [target_sig[(z, t)] for t in TYPES]
        s = sum(probs) or 1.0
        probs = [p / s for p in probs]
        counts_arr = rng.multinomial(z_total, probs)
        composition[z] = {t: int(c) for t, c in zip(TYPES, counts_arr)}
    
    # ── Step 3: 감 위치 먼저 결정 ──
    n_gam_total = sum(composition[z]['Gam'] for z in ZONES)
    
    # 감 배치: 세그먼트 단위로 분산
    # 평균 간격 = length / n_gam_total
    avg_gap = max(2, length // max(n_gam_total, 1))
    
    gam_positions = []
    pos = rng.integers(1, max(2, avg_gap))
    while pos < length and len(gam_positions) < n_gam_total:
        gam_positions.append(pos)
        # 간격에 약간의 변이 추가 (자연스러운 분포)
        gap = max(1, int(avg_gap + rng.normal(0, avg_gap * 0.3)))
        pos += gap
    
    # 부족한 감 위치 채우기
    while len(gam_positions) < n_gam_total:
        new_pos = rng.integers(0, length)
        if new_pos not in gam_positions:
            gam_positions.append(new_pos)
    gam_positions = sorted(gam_positions[:n_gam_total])
    
    # 감 종류 결정: P(꺾임), G(유연), C(고정)
    # C는 양쪽 끝에, P는 중간에, G는 나머지
    gam_aas = []
    for i, pos in enumerate(gam_positions):
        rel_pos = pos / length
        if rel_pos < 0.15 or rel_pos > 0.85:
            gam_aas.append('C')  # 양끝: SS-bond 후보
        elif i > 0 and gam_positions[i] - gam_positions[i-1] > avg_gap * 1.2:
            gam_aas.append('P')  # 긴 간격 후: 강한 꺾임
        else:
            gam_aas.append('G')  # 기본: 유연
    
    # ── Step 4: 세그먼트 식별 ──
    segments = []
    prev = 0
    for gp in gam_positions:
        if gp > prev:
            segments.append((prev, gp))
        prev = gp + 1
    if prev < length:
        segments.append((prev, length))
    
    # ── Step 5: 서열 배열 초기화 ──
    sequence = ['X'] * length
    seq_types = [None] * length
    
    # 감 배치
    for pos, aa in zip(gam_positions, gam_aas):
        sequence[pos] = aa
        seq_types[pos] = 'Gam'
    
    # ── Step 6: 세그먼트별 곤/리/건 충전 ──
    # 중앙 세그먼트 → 곤 우세, 양끝 세그먼트 → 건 우세
    n_segments = len(segments)
    
    # Zone별 남은 곤/리/건 개수
    remaining_counts = {t: sum(composition[z][t] for z in ZONES) for t in ['Gon', 'Ri', 'Geon']}
    
    for seg_idx, (start, end) in enumerate(segments):
        seg_len = end - start
        empty_positions = [p for p in range(start, end) if sequence[p] == 'X']
        
        if not empty_positions:
            continue
        
        n_empty = len(empty_positions)
        
        # 세그먼트 중심성: 0=양끝, 1=중앙
        if n_segments > 1:
            centrality = 1.0 - abs(2 * seg_idx / (n_segments - 1) - 1)
        else:
            centrality = 0.5
        
        # 중심 세그먼트: 곤 비율 높음, 양끝: 건 비율 높음
        gon_frac = 0.3 + 0.2 * centrality    # 0.3~0.5
        geon_frac = 0.15 + 0.15 * (1 - centrality)  # 0.15~0.30
        ri_frac = 1.0 - gon_frac - geon_frac  # 나머지
        
        n_gon = max(0, min(remaining_counts['Gon'], round(n_empty * gon_frac)))
        n_geon = max(0, min(remaining_counts['Geon'], round(n_empty * geon_frac)))
        n_ri = max(0, min(remaining_counts['Ri'], n_empty - n_gon - n_geon))
        
        # 남은 자리는 가장 많이 남은 타입으로
        n_total = n_gon + n_geon + n_ri
        if n_total < n_empty:
            diff = n_empty - n_total
            if remaining_counts['Ri'] - n_ri > 0:
                n_ri += min(diff, remaining_counts['Ri'] - n_ri)
                diff = n_empty - n_gon - n_geon - n_ri
            if diff > 0 and remaining_counts['Gon'] - n_gon > 0:
                n_gon += min(diff, remaining_counts['Gon'] - n_gon)
                diff = n_empty - n_gon - n_geon - n_ri
            if diff > 0 and remaining_counts['Geon'] - n_geon > 0:
                n_geon += min(diff, remaining_counts['Geon'] - n_geon)
        
        # 곤-건 최대 거리: 곤을 세그먼트 중앙에, 건을 양끝에
        mid = len(empty_positions) // 2
        
        # 위치 배정
        types_to_place = []
        types_to_place.extend(['Gon'] * n_gon)
        types_to_place.extend(['Ri'] * n_ri)
        types_to_place.extend(['Geon'] * n_geon)
        
        # 곤: 중앙부터, 건: 양끝부터, 리: 사이
        sorted_positions = sorted(empty_positions, key=lambda p: abs(p - (start + end) / 2))
        
        # 곤을 중앙에
        gon_positions = sorted_positions[:n_gon]
        # 건을 양끝에
        geon_positions = sorted_positions[-(n_geon):] if n_geon > 0 else []
        # 리를 사이에
        ri_positions = [p for p in sorted_positions[n_gon:] if p not in geon_positions][:n_ri]
        
        # 아미노산 선택 및 배치
        aa_pools = {'Gon': GON_AAS, 'Ri': RI_AAS, 'Geon': GEON_AAS}
        
        for pos in gon_positions:
            aa = GON_AAS[rng.integers(0, len(GON_AAS))]
            sequence[pos] = aa
            seq_types[pos] = 'Gon'
            remaining_counts['Gon'] -= 1
        
        for pos in ri_positions:
            aa = RI_AAS[rng.integers(0, len(RI_AAS))]
            sequence[pos] = aa
            seq_types[pos] = 'Ri'
            remaining_counts['Ri'] -= 1
        
        for pos in geon_positions:
            aa = GEON_AAS[rng.integers(0, len(GEON_AAS))]
            sequence[pos] = aa
            seq_types[pos] = 'Geon'
            remaining_counts['Geon'] -= 1
    
    # ── Step 7: 남은 빈자리 채우기 ──
    for i in range(length):
        if sequence[i] == 'X':
            # 남은 개수가 가장 많은 타입
            if remaining_counts['Gon'] > 0:
                sequence[i] = GON_AAS[rng.integers(0, len(GON_AAS))]
                seq_types[i] = 'Gon'
                remaining_counts['Gon'] -= 1
            elif remaining_counts['Ri'] > 0:
                sequence[i] = RI_AAS[rng.integers(0, len(RI_AAS))]
                seq_types[i] = 'Ri'
                remaining_counts['Ri'] -= 1
            elif remaining_counts['Geon'] > 0:
                sequence[i] = GEON_AAS[rng.integers(0, len(GEON_AAS))]
                seq_types[i] = 'Geon'
                remaining_counts['Geon'] -= 1
            else:
                sequence[i] = 'A'
                seq_types[i] = 'Gon'
    
    # ── Step 8: 인접 친화도 최적화 (국소 스왑) ──
    seq_arr = list(sequence)
    for _ in range(length * 2):
        i = rng.integers(0, length - 1)
        j = rng.integers(0, length - 1)
        if i == j or seq_types[i] == 'Gam' or seq_types[j] == 'Gam':
            continue
        if seq_types[i] == seq_types[j]:
            continue
        
        # 현재 친화도
        curr_score = 0
        for k in [i, j]:
            if k > 0:
                curr_score += AFFINITY.get((seq_types[k], seq_types[k-1]), 0)
            if k < length - 1:
                curr_score += AFFINITY.get((seq_types[k], seq_types[k+1]), 0)
        
        # 스왑
        seq_arr[i], seq_arr[j] = seq_arr[j], seq_arr[i]
        seq_types[i], seq_types[j] = seq_types[j], seq_types[i]
        
        new_score = 0
        for k in [i, j]:
            if k > 0:
                new_score += AFFINITY.get((seq_types[k], seq_types[k-1]), 0)
            if k < length - 1:
                new_score += AFFINITY.get((seq_types[k], seq_types[k+1]), 0)
        
        if new_score < curr_score:
            # 스왑 취소
            seq_arr[i], seq_arr[j] = seq_arr[j], seq_arr[i]
            seq_types[i], seq_types[j] = seq_types[j], seq_types[i]
    
    final_seq = ''.join(seq_arr)
    
    # ── 분석 ──
    type_counts = Counter(seq_types)
    total = sum(type_counts.values())
    type_ratios = {t: type_counts.get(t, 0) / total for t in TYPES}
    
    # 곤-건 직접 인접
    gon_geon_adj = 0
    for i in range(len(final_seq) - 1):
        t1, t2 = seq_types[i], seq_types[i+1]
        if (t1 == 'Gon' and t2 == 'Geon') or (t1 == 'Geon' and t2 == 'Gon'):
            gon_geon_adj += 1
    
    # 감 밀도
    actual_gam_density = type_counts.get('Gam', 0) / total
    
    # 인접 친화도 평균
    adj_scores = []
    for i in range(len(final_seq) - 1):
        adj_scores.append(AFFINITY.get((seq_types[i], seq_types[i+1]), 0))
    avg_affinity = np.mean(adj_scores) if adj_scores else 0
    
    metadata = {
        'length': length,
        'target_antagonism': target_antagonism,
        'type_ratios': type_ratios,
        'gam_density': actual_gam_density,
        'gon_geon_adjacent': gon_geon_adj,
        'avg_affinity': avg_affinity,
        'n_segments': n_segments,
        'gam_positions': gam_positions,
    }
    
    return final_seq, target_sig, metadata


print("✅ Creator Engine v4 정의 완료")
print("   감 선배치 → 세그먼트 충전 → 곤-건 최대 거리 → 인접 최적화")


## 4. 테스트 서열 생성

In [ ]:
# ══════════════════════════════════════════
# 5가지 유형별 테스트 서열 생성
# ══════════════════════════════════════════

test_cases = [
    {"name": "T1_Globular_S",   "L": 100, "A": 0.50, "desc": "소형 구형"},
    {"name": "T1_Globular_M",   "L": 150, "A": 0.55, "desc": "중형 고안정"},
    {"name": "T2_Enzyme",       "L": 150, "A": 0.30, "desc": "효소형"},
    {"name": "T3_Balanced",     "L": 200, "A": 0.15, "desc": "균형형"},
    {"name": "T5_Special",      "L": 120, "A": 0.05, "desc": "특수형"},
]

# 각 유형 3개씩 (seed 변경)
all_sequences = []

print("🔧 Creator Engine v4 서열 생성")
print("=" * 70)

for tc in test_cases:
    print(f"\n  {tc['name']} ({tc['desc']}): L={tc['L']}, A={tc['A']}")
    
    for trial in range(3):
        seq, sig, meta = creator_engine_v4(
            length=tc['L'],
            target_antagonism=tc['A'],
            seed=42 + trial * 100
        )
        
        entry = {
            'name': f"{tc['name']}_{trial+1}",
            'type': tc['name'],
            'desc': tc['desc'],
            'sequence': seq,
            'signature': sig,
            'metadata': meta,
        }
        all_sequences.append(entry)
        
        m = meta
        print(f"    #{trial+1}: 곤={m['type_ratios']['Gon']:.0%} 감={m['gam_density']:.0%} "
              f"리={m['type_ratios']['Ri']:.0%} 건={m['type_ratios']['Geon']:.0%} "
              f"곤건인접={m['gon_geon_adjacent']} 친화도={m['avg_affinity']:.2f}")

# 자연 단백질 대조군
natural_proteins = {
    'Ubiquitin': 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGG',
    'Lysozyme': 'KVFGRCELAAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRNTDGSTDYGILQINSRWWCNDGRTPGSRNLCNIPCSALLSSDITASVNCAKKIVSDGNGMNAWVAWRNRCKGTDVQAWIRGCRL',
    'Villin': 'MLSDEDFKAVFGMTRSAFANLPLWKQQNLKKEKGLF',
    'TrpCage': 'NLYIQWLKDGGPSSGRPPPS',
    'Crambin': 'TTCCPSIVARSNFNVCRLPGTPEALCATYTGCIIIPGATCPGDYAN',
}

for name, seq in natural_proteins.items():
    all_sequences.append({
        'name': f'Natural_{name}',
        'type': 'Natural',
        'desc': '자연 단백질 (대조군)',
        'sequence': seq,
        'signature': None,
        'metadata': {'length': len(seq), 'target_antagonism': None},
    })

print(f"\n✅ 총 {len(all_sequences)}개 서열 생성 ({len(test_cases)*3} v4 + {len(natural_proteins)} 자연)")


## 5. ESMFold 구조 예측

HuggingFace Transformers의 `EsmForProteinFolding`을 사용합니다.
openfold 의존성이 없어 Colab에서 바로 실행됩니다.


In [ ]:
from transformers import AutoTokenizer, EsmForProteinFolding

print("🔄 ESMFold 모델 로딩 중... (첫 실행 시 ~3분)")
tokenizer = AutoTokenizer.from_pretrained("facebook/esmfold_v1")
model = EsmForProteinFolding.from_pretrained("facebook/esmfold_v1", low_cpu_mem_usage=True)

model = model.to(device)
model.eval()

# 메모리 최적화
model.esm = model.esm.half()
torch.backends.cuda.matmul.allow_tf32 = True

print("✅ ESMFold 로드 완료")


In [ ]:
def predict_structure(sequence, max_length=400):
    """ESMFold로 구조 예측 → pLDDT 반환."""
    
    # 길이 제한 (GPU 메모리)
    seq = sequence[:max_length]
    
    with torch.no_grad():
        tokenized = tokenizer([seq], return_tensors="pt", 
                             add_special_tokens=False).to(device)
        
        try:
            output = model(**tokenized)
            
            # pLDDT 추출
            plddt = output.plddt.cpu().numpy().flatten()[:len(seq)]
            
            # PDB 문자열 (선택)
            pdb_string = None
            try:
                from transformers.models.esm.openfold_utils import OFProtein
                pdb_string = model.output_to_pdb(output)[0]
            except:
                pass
            
            return {
                'plddt_mean': float(np.mean(plddt)),
                'plddt_median': float(np.median(plddt)),
                'plddt_per_residue': plddt.tolist(),
                'plddt_above70': float(np.mean(plddt > 70)),
                'plddt_above50': float(np.mean(plddt > 50)),
                'pdb_string': pdb_string,
                'length': len(seq),
            }
            
        except RuntimeError as e:
            if "out of memory" in str(e):
                torch.cuda.empty_cache()
                return {'error': 'OOM', 'length': len(seq)}
            raise e


print("✅ predict_structure 함수 정의 완료")


## 6. 교차 검증 실행

In [ ]:
# ══════════════════════════════════════════
# 전체 서열 ESMFold 검증
# ══════════════════════════════════════════

print("🔬 ESMFold 교차 검증 시작")
print("=" * 70)

results = []

for i, entry in enumerate(all_sequences):
    name = entry['name']
    seq = entry['sequence']
    
    print(f"\n  [{i+1}/{len(all_sequences)}] {name} (L={len(seq)})", end=" → ")
    
    pred = predict_structure(seq)
    
    if 'error' in pred:
        print(f"❌ {pred['error']}")
        results.append({**entry, 'prediction': pred})
        continue
    
    plddt = pred['plddt_mean']
    above70 = pred['plddt_above70']
    
    status = "✅" if plddt > 70 else ("⚠" if plddt > 50 else "❌")
    print(f"{status} pLDDT={plddt:.1f}, >70: {above70:.0%}")
    
    entry['prediction'] = pred
    results.append(entry)
    
    # GPU 메모리 정리
    if i % 5 == 0:
        torch.cuda.empty_cache()

print(f"\n{'=' * 70}")
print(f"✅ 검증 완료: {len(results)}개")


## 7. 결과 분석

In [ ]:
import matplotlib.pyplot as plt

# ══════════════════════════════════════════
# 결과 정리
# ══════════════════════════════════════════

print("\n" + "=" * 70)
print("  UFoE Creator Engine v4 + ESMFold 교차 검증 결과")
print("=" * 70)

# v4 생성 vs 자연 단백질
v4_results = [r for r in results if r['type'] != 'Natural' and 'error' not in r.get('prediction', {})]
nat_results = [r for r in results if r['type'] == 'Natural' and 'error' not in r.get('prediction', {})]

print(f"\n  v4 생성 서열: {len(v4_results)}개")
print(f"  자연 대조군: {len(nat_results)}개")

# 유형별 pLDDT
print(f"\n  {'유형':<20} {'n':>3}  {'pLDDT':>6}  {'>70':>5}  {'판정'}")
print(f"  {'─'*55}")

type_groups = defaultdict(list)
for r in v4_results:
    type_groups[r['type']].append(r['prediction']['plddt_mean'])

for tp, plddts in sorted(type_groups.items()):
    mean_p = np.mean(plddts)
    above70 = np.mean([p > 70 for p in plddts])
    status = "✅" if mean_p > 70 else ("⚠" if mean_p > 50 else "❌")
    print(f"  {tp:<20} {len(plddts):>3}  {mean_p:>5.1f}  {above70:>4.0%}  {status}")

if nat_results:
    nat_plddts = [r['prediction']['plddt_mean'] for r in nat_results]
    print(f"  {'Natural (대조군)':<20} {len(nat_plddts):>3}  {np.mean(nat_plddts):>5.1f}  "
          f"{np.mean([p > 70 for p in nat_plddts]):>4.0%}  ✅")

# 전체 통계
all_v4_plddt = [r['prediction']['plddt_mean'] for r in v4_results]
all_nat_plddt = [r['prediction']['plddt_mean'] for r in nat_results] if nat_results else []

print(f"\n  v4 전체 평균 pLDDT: {np.mean(all_v4_plddt):.1f} ± {np.std(all_v4_plddt):.1f}")
if all_nat_plddt:
    print(f"  자연 전체 평균 pLDDT: {np.mean(all_nat_plddt):.1f} ± {np.std(all_nat_plddt):.1f}")

v4_pass = np.mean([p > 70 for p in all_v4_plddt])
print(f"  v4 pLDDT > 70 통과율: {v4_pass:.0%}")

# ══════════════════════════════════════════
# 시각화
# ══════════════════════════════════════════

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 1. pLDDT 분포
ax = axes[0]
if all_v4_plddt:
    ax.hist(all_v4_plddt, bins=15, alpha=0.7, label=f'v4 (n={len(all_v4_plddt)})', color='#FF6B35')
if all_nat_plddt:
    ax.hist(all_nat_plddt, bins=8, alpha=0.7, label=f'Natural (n={len(all_nat_plddt)})', color='#2196F3')
ax.axvline(x=70, color='red', linestyle='--', alpha=0.5, label='pLDDT=70')
ax.set_xlabel('Mean pLDDT')
ax.set_ylabel('Count')
ax.set_title('pLDDT Distribution')
ax.legend()

# 2. 유형별 pLDDT
ax = axes[1]
types_list = list(type_groups.keys()) + (['Natural'] if nat_results else [])
plddts_list = [type_groups[t] for t in type_groups.keys()]
if nat_results:
    plddts_list.append(all_nat_plddt)
colors = ['#FF6B35'] * len(type_groups) + ['#2196F3'] * (1 if nat_results else 0)
bp = ax.boxplot(plddts_list, labels=[t.split('_')[0] for t in types_list], 
                patch_artist=True)
for patch, color in zip(bp['boxes'], colors):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)
ax.axhline(y=70, color='red', linestyle='--', alpha=0.5)
ax.set_ylabel('pLDDT')
ax.set_title('pLDDT by Type')
ax.tick_params(axis='x', rotation=45)

# 3. 곤-건 인접 vs pLDDT
ax = axes[2]
for r in v4_results:
    m = r['metadata']
    p = r['prediction']['plddt_mean']
    ax.scatter(m.get('gon_geon_adjacent', 0), p, 
              c='#FF6B35', alpha=0.7, s=50)
ax.set_xlabel('Gon-Geon Adjacent Count')
ax.set_ylabel('pLDDT')
ax.set_title('Gon-Geon Adjacency vs pLDDT')
ax.axhline(y=70, color='red', linestyle='--', alpha=0.5)

plt.suptitle('UFoE Creator Engine v4 — ESMFold Validation', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('ufoe_v4_esmfold_results.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n💾 ufoe_v4_esmfold_results.png")


## 8. 12칸 매트릭스 보존 확인

In [ ]:
# ══════════════════════════════════════════
# ESMFold pLDDT Zone별 분석
# → 감 위치의 pLDDT가 턴 예측과 일치하는지
# ══════════════════════════════════════════

print("\n" + "=" * 70)
print("  감 결절점 — ESMFold pLDDT 분석")
print("=" * 70)

for r in v4_results[:5]:  # 처음 5개만
    pred = r['prediction']
    seq = r['sequence']
    plddt = pred['plddt_per_residue']
    
    if len(plddt) < len(seq):
        seq = seq[:len(plddt)]
    
    # 감 위치 vs 비감 위치의 pLDDT
    gam_plddt = [plddt[i] for i in range(len(seq)) if seq[i] in 'CGP']
    non_gam_plddt = [plddt[i] for i in range(len(seq)) if seq[i] not in 'CGP']
    
    # 감 위치 pLDDT가 낮으면 → 루프/턴 (유연한 영역)
    # 감 위치 pLDDT가 높으면 → 구조적으로 잘 정의된 턴
    
    print(f"\n  {r['name']}:")
    print(f"    감 pLDDT:   {np.mean(gam_plddt):.1f} ± {np.std(gam_plddt):.1f}")
    print(f"    비감 pLDDT: {np.mean(non_gam_plddt):.1f} ± {np.std(non_gam_plddt):.1f}")
    
    # pLDDT 프로파일에서 감 위치 표시
    diff = np.mean(gam_plddt) - np.mean(non_gam_plddt)
    if diff < -5:
        print(f"    → 감 위치가 더 유연 (Δ={diff:.1f}) — 턴/루프 역할 확인")
    elif diff > 5:
        print(f"    → 감 위치가 더 안정 (Δ={diff:+.1f}) — 구조적 앵커 역할")
    else:
        print(f"    → 감-비감 유사 (Δ={diff:+.1f})")

print(f"\n{'=' * 70}")
print(f"  판정 기준:")
print(f"  - 감 위치의 pLDDT가 비감보다 낮으면: 감이 유연한 턴/루프에 위치 (기대)")
print(f"  - 전체 pLDDT > 70이면: v4 설계 서열이 안정적으로 접힘 (목표)")
print(f"{'=' * 70}")


## 9. 결과 저장

In [ ]:
# JSON 저장 (서열 + pLDDT)
save_data = []
for r in results:
    entry = {
        'name': r['name'],
        'type': r['type'],
        'sequence': r['sequence'],
        'length': len(r['sequence']),
    }
    
    if 'prediction' in r and 'error' not in r['prediction']:
        entry['plddt_mean'] = r['prediction']['plddt_mean']
        entry['plddt_above70'] = r['prediction']['plddt_above70']
    
    if r.get('metadata'):
        m = r['metadata']
        entry['target_antagonism'] = m.get('target_antagonism')
        entry['gam_density'] = m.get('gam_density')
        entry['gon_geon_adjacent'] = m.get('gon_geon_adjacent')
    
    save_data.append(entry)

with open('ufoe_v4_esmfold_results.json', 'w') as f:
    json.dump(save_data, f, indent=2)

print("💾 ufoe_v4_esmfold_results.json")
print("💾 ufoe_v4_esmfold_results.png")

# PDB 저장 (접힌 구조)
pdb_count = 0
for r in results:
    if r.get('prediction', {}).get('pdb_string'):
        fname = f"ufoe_{r['name']}.pdb"
        with open(fname, 'w') as f:
            f.write(r['prediction']['pdb_string'])
        pdb_count += 1

if pdb_count:
    print(f"💾 {pdb_count}개 PDB 파일 저장")

print(f"\n{'═' * 70}")
print(f"  🧬 UFoE Creator Engine v4 + ESMFold 교차 검증 완료")
print(f"  {len(v4_results)}개 v4 서열 + {len(nat_results)}개 자연 대조군")
print(f"  v4 평균 pLDDT: {np.mean(all_v4_plddt):.1f}")
print(f"  v4 pLDDT>70 통과율: {v4_pass:.0%}")
print(f"{'═' * 70}")
